# Домашняя работа по теме "Машинное обучение ранжированию"

В этом ДЗ мы:
- научимся работать со стандартным датасетом для машинного обучения ранжированию [MSLR](https://www.microsoft.com/en-us/research/project/mslr/)
- попробуем применить на практике все то, чему мы научились на семинаре

## Как будет происходить сдача ДЗ

Вам надо:
- форкнуть эту репу
- создать бранч в котором вы дальше будете работать
- реализовать класс Model в этом ноутбуке
- убедиться, что ваша реализация выбивает NDCG@10 выше бейзлайна (см. ниже)
- запушить ваш бранч и поставить Pull Request
- в комментарии написать какой скор вы выбили

В таком случае мы (организаторы):

- счекаутим вашу бранчу
- проверим что ваша реализация действительно выбивает заявленный скор

Предполагается, что и вы, и мы работаем в виртаульном окружении как в семинаре про машинное обучение ранжированию: seminars/7-learning-to-rank/requirements.txt(подробнее про работу с виртуальными окружениями README в корне этой репы).

Оценка:
- За выбитый скор больше **0.507** назначаем **5** баллов, за скор больше (или равно) **0.510** назначаем максимальный балл -- 10 баллов
- Тот из участников кто выбъет самый высокий скор получит еще +10 баллов

При сдаче кода важно помнить о том, что:
- В коде не должно быть захардкоженных с потолка взятых гиперпараметров (таких как число деревьев, learning rate и т.п.) -- обязательно должен быть представлен код который их подбирает!
- Решение должно быть стабильно от запуска к запуску (на CPU) т.е. все seed'ы для генераторов случайных чисел должны быть фиксированы
- Мы (организаторы) будем запускать код на CPU поэтому, даже если вы использовали для подбора параметров GPU, финальный скор надо репортить на CPU

## Пререквизиты

Импортируем все что нам понадобится для дальнейшей работы:

In [1]:
import pathlib
from timeit import default_timer as timer

import numpy as np
import pandas as pd
from tqdm import tqdm
import catboost
import copy
import random
import pickle

from catboost import datasets, utils

## Датасет MSLR (Microsoft Learning to Rank)

Загрузим датасет MSLR.

Полный датасет можно скачать с официального сайта: https://www.microsoft.com/en-us/research/project/mslr/

Строго говоря, он состоит их 2х частей:

- основной датасет MSLR-WEB30K -- он содержит более 30 тыс. запросов
- "маленький" датасет MSLR-WEB10K, который содержит только 10 тыс. запросов и является случайным сэмплом датасета MSLR-WEB30K

в этом ДЗ мы будем работать с MSLR-WEB10K, т.к. полная версия датасета может просто не поместиться у нас в RAM (и, тем более, в память видеокарты если мы учимся на GPU)

Будем считать, что мы самостоятельно скачали датасет MSLR-WEB10K с официального сайта, поместили его в папку КОРЕНЬ-ЭТОЙ-РЕПЫ/data/mslr-web10k и раззиповали.

В результате у нас должна получиться следующая структура папок:

In [2]:
!ls /kaggle/input/vk-hw3-ranking/MSLR-WEB10K

Fold1


Заметим, что датасет довольно большой, в распакованном виде он весит 7.7 GB.

Датасет состоит из нескольких фолдов, которые по сути представляют из себя разные разбиения одних и тех же данных на обучающее, валидационное и тестовые множеста.

Дальше мы будем использовать только первый фолд: Fold1.

Заглянем внутрь:

In [3]:
!ls -lh /kaggle/input/vk-hw3-ranking/MSLR-WEB10K/Fold1

total 1.3G
-rw-r--r-- 1 nobody nogroup 267M Apr  8 15:08 test.txt
-rw-r--r-- 1 nobody nogroup 800M Apr  8 15:08 train.txt
-rw-r--r-- 1 nobody nogroup 261M Apr  8 15:08 vali.txt


Видим, что у нас 3 файла с говорящими названиями, соответсвующими сплитам нашего датасета.

Посмотрим на содержимое одного из файлов:

In [4]:
!head -n 1 /kaggle/input/vk-hw3-ranking/MSLR-WEB10K/Fold1/train.txt

2 qid:1 1:3 2:3 3:0 4:0 5:3 6:1 7:1 8:0 9:0 10:1 11:156 12:4 13:0 14:7 15:167 16:6.931275 17:22.076928 18:19.673353 19:22.255383 20:6.926551 21:3 22:3 23:0 24:0 25:6 26:1 27:1 28:0 29:0 30:2 31:1 32:1 33:0 34:0 35:2 36:1 37:1 38:0 39:0 40:2 41:0 42:0 43:0 44:0 45:0 46:0.019231 47:0.75000 48:0 49:0 50:0.035928 51:0.00641 52:0.25000 53:0 54:0 55:0.011976 56:0.00641 57:0.25000 58:0 59:0 60:0.011976 61:0.00641 62:0.25000 63:0 64:0 65:0.011976 66:0 67:0 68:0 69:0 70:0 71:6.931275 72:22.076928 73:0 74:0 75:13.853103 76:1.152128 77:5.99246 78:0 79:0 80:2.297197 81:3.078917 82:8.517343 83:0 84:0 85:6.156595 86:2.310425 87:7.358976 88:0 89:0 90:4.617701 91:0.694726 92:1.084169 93:0 94:0 95:2.78795 96:1 97:1 98:0 99:0 100:1 101:1 102:1 103:0 104:0 105:1 106:12.941469 107:20.59276 108:0 109:0 110:16.766961 111:-18.567793 112:-7.760072 113:-20.838749 114:-25.436074 115:-14.518523 116:-21.710022 117:-21.339609 118:-24.497864 119:-27.690319 120:-20.203779 121:-15.449379 122:-4.474452 123:-23.634899 

Видим, что данные лежат в уже знакомом нам по семинару формате:

- В первой колонке лежит таргет (оценка асессора), по 5-балльной шкале релевантности: от 0 до 4 (включительно)
- Во второй колонке лежит ID запроса, по которому можно сгруппировать все оценки документов в рамках одного и того же запроса
- Дальше идет вектор из 128 фичей (таких как значения BM25 и т.п.), их точная природа нам сейчас на важна

В файле qid и все-фичи кодируются в формате КЛЮЧ:ЗНАЧЕНИЕ, напр. 130:116 -- тут 130 это номер фичи, а 116 -- ее значение.

Такой формат в мире машинного обучения часто называют svm light формат (в честь когда-то популярной библиотеки SVM-Light)

Напишем немного вспомогательного кода для загрузки этого датасета:

In [5]:
def generate_column_names(num_features):
    """Generates column names for LETOR-like datasets"""
    columns = ['label', 'qid']
    for i in range(num_features):
        column = f"feature_{i+1}"
        columns.append(column)
    return columns
    
def load_svmlight_file(input_file, max_num_lines=0):
    """Loads dataset split in SVM-Light format"""
    def _parse_field(field):
        parts = field.split(':')
        if len(parts) != 2:
            raise Exception(f"invalid number of parts in field {field}")
        return parts

    num_features = 136
    exp_num_fields = num_features + 2
    num_lines = 0
    X = []
    with open(input_file, 'rt') as f:
        for line in f:
            try:
                num_lines += 1
                                  
                # Parse into fields
                fields = line.rstrip().split(' ')
                num_fields = len(fields)
                if num_fields != exp_num_fields:
                    raise Exception(f"invalid number of fields {num_fields}")
    
                # Parse every field
                x = np.zeros(exp_num_fields, dtype=np.float32)
                label = int(fields[0])
                x[0] = label
                _, qid_str = _parse_field(fields[1])
                qid = int(qid_str)
                x[1] = qid
                for i, field in enumerate(fields[2:]):
                    _, feature_str = _parse_field(field)
                    x[i+2] = float(feature_str)
    
                # Add new object
                X.append(x)
                if num_lines % 50000 == 0:
                    print(f"Loaded {num_lines} lines...")
                if max_num_lines > 0 and num_lines == max_num_lines:
                    print(f"WARNING: stop loading, line limit reached: max_num_lines = {max_num_lines} input_file = {input_file}")
                    break
            except Exception as e:
                raise Exception(f"error at line {num_lines} in {input_file}") from e
    
    # To pandas
    print(f'Creating pandas dataframe with size = {len(X)}')
    df = pd.DataFrame(X, columns=generate_column_names(num_features))
    print(f"Loaded SVM-Light file {input_file}")
    return df

И теперь загрузим датасет:

In [6]:
fold_dir = pathlib.Path("/kaggle/input/vk-hw3-ranking/MSLR-WEB10K/Fold1")

df_train = load_svmlight_file(fold_dir.joinpath("train.txt"))
df_valid = load_svmlight_file(fold_dir.joinpath("vali.txt"))
df_test = load_svmlight_file(fold_dir.joinpath("test.txt"))
print(f"Dataset loaded from fold_dir {fold_dir}")

Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded 250000 lines...
Loaded 300000 lines...
Loaded 350000 lines...
Loaded 400000 lines...
Loaded 450000 lines...
Loaded 500000 lines...
Loaded 550000 lines...
Loaded 600000 lines...
Loaded 650000 lines...
Loaded 700000 lines...
Creating pandas dataframe with size = 723412
Loaded SVM-Light file /kaggle/input/vk-hw3-ranking/MSLR-WEB10K/Fold1/train.txt
Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Creating pandas dataframe with size = 235259
Loaded SVM-Light file /kaggle/input/vk-hw3-ranking/MSLR-WEB10K/Fold1/vali.txt
Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Creating pandas dataframe with size = 241521
Loaded SVM-Light file /kaggle/input/vk-hw3-ranking/MSLR-WEB10K/Fold1/test.txt
Dataset loaded from fold_dir /kaggle/input/vk-hw3-ranking/MSLR-WEB10K/Fold1


Посмотрим на данные:

In [7]:
df_train.head(5)

,label,qid,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_127,feature_128,feature_129,feature_130,feature_131,feature_132,feature_133,feature_134,feature_135,feature_136
0,2.0,1.0,3.0,3.0,0.0,0.0,3.0,1.0,1.0,0.000000,...,62.0,11089534.0,2.0,116.0,64034.0,13.0,3.0,0.0,0.0,0.0
1,2.0,1.0,3.0,0.0,3.0,0.0,3.0,1.0,0.0,1.000000,...,54.0,11089534.0,2.0,124.0,64034.0,1.0,2.0,0.0,0.0,0.0
2,0.0,1.0,3.0,0.0,2.0,0.0,3.0,1.0,0.0,0.666667,...,45.0,3.0,1.0,124.0,3344.0,14.0,67.0,0.0,0.0,0.0
3,2.0,1.0,3.0,0.0,3.0,0.0,3.0,1.0,0.0,1.000000,...,56.0,11089534.0,13.0,123.0,63933.0,1.0,3.0,0.0,0.0,0.0
4,1.0,1.0,3.0,0.0,3.0,0.0,3.0,1.0,0.0,1.000000,...,64.0,5.0,7.0,256.0,49697.0,1.0,13.0,0.0,0.0,0.0


Т.е. теперь мы видим что данные доступны в точно таком же виде, как это было в семинаре.

Проведем небольшой EDA.

Всего у нас 723 тыс. документов в трейне:

In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 723412 entries, 0 to 723411
Columns: 138 entries, label to feature_136
dtypes: float32(138)
memory usage: 380.8 MB


235 тыс. документов в валидации:

In [9]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235259 entries, 0 to 235258
Columns: 138 entries, label to feature_136
dtypes: float32(138)
memory usage: 123.8 MB


И 241 тыс. документов в тесте:

In [10]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241521 entries, 0 to 241520
Columns: 138 entries, label to feature_136
dtypes: float32(138)
memory usage: 127.1 MB


Сколько у нас запросов?

In [11]:
num_queries_train = df_train['qid'].nunique()
num_queries_valid = df_valid['qid'].nunique()
num_queries_test = df_test['qid'].nunique()
print(f"Got {num_queries_train} train, {num_queries_valid} valid and {num_queries_test} test queries")

Got 6000 train, 2000 valid and 2000 test queries


## Обучаем модель

In [12]:
def to_catboost_dataset(df):
    y = df['label'].to_numpy()                       # Label: [0-4]
    q = df['qid'].to_numpy().astype('uint32')        # Query Id
    X = df.drop(columns=['label', 'qid']).to_numpy() # 136 features
    return (X, y, q)

X_train, y_train, q_train = to_catboost_dataset(df_train)
X_valid, y_valid, q_valid = to_catboost_dataset(df_valid)
X_test, y_test, q_test = to_catboost_dataset(df_test)

In [13]:
pool_train = catboost.Pool(data=X_train, label=y_train, group_id=q_train)
pool_valid = catboost.Pool(data=X_valid, label=y_valid, group_id=q_valid)

In [28]:
def search(n_iter, pool_train, pool_valid, X_test, y_test):
    np.random.seed(0)
    random.seed(0)
    best_score = 0
    best_model = None
    best_params = {}
    for i in range(n_iter):
        print(f'{"-"*20} Model number {i+1} {"-"*20}')
        params = {
                'learning_rate': np.random.uniform(1e-3, 1e-1),
                'depth': random.randint(3, 8),
                'l2_leaf_reg': np.random.uniform(1, 30),
                'loss_function': 'YetiRank',
                'eval_metric': 'NDCG:top=10;type=Exp',
                'iterations': 1000,
                'bagging_temperature': np.random.uniform(0.7, 1),
                'use_best_model': True,
                'min_data_in_leaf': random.randint(1, 100),
                'verbose': 200,
                'grow_policy': random.choice(['SymmetricTree', 'Depthwise', 'Lossguide']),
                #'max_leaves': random.randint(10, 50),
                'task_type': "GPU",
               }
        print('Params:')
        for k,v in params.items():
            print(f'\t - {k}: {v}')
        model = catboost.CatBoost(params)
        start = timer()
        model.fit(pool_train, eval_set=pool_valid)
        y_pred = model.predict(X_test)
        score = utils.eval_metric(y_test, y_pred, 'NDCG:top=10;type=Exp', group_id=q_test)[0]
        if score > best_score:
            best_model = model
            best_score = score
            best_params = params
            
        elapsed = timer() - start
        print(f"Model fit: elapsed = {elapsed:.3f} with test/NDCG@10 = {score}")
        
    return best_model, best_params

In [30]:
best_model, best_params = search(25, pool_train, pool_valid, X_test, y_test)

-------------------- Model number 1 --------------------
Params:
	 - learning_rate: 0.055332536888805156
	 - depth: 6
	 - l2_leaf_reg: 21.740491624800164
	 - loss_function: YetiRank
	 - eval_metric: NDCG:top=10;type=Exp
	 - iterations: 1000
	 - bagging_temperature: 0.8808290128214932
	 - use_best_model: True
	 - min_data_in_leaf: 98
	 - verbose: 200
	 - grow_policy: Depthwise
	 - task_type: GPU


Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.3312943	best: 0.3312943 (0)	total: 52.1ms	remaining: 52.1s
200:	test: 0.4809622	best: 0.4809622 (200)	total: 6.45s	remaining: 25.6s
400:	test: 0.4950912	best: 0.4950912 (400)	total: 12.8s	remaining: 19.1s
600:	test: 0.5006796	best: 0.5007134 (595)	total: 19.1s	remaining: 12.7s
800:	test: 0.5038547	best: 0.5039248 (795)	total: 25.4s	remaining: 6.3s
999:	test: 0.5063628	best: 0.5064785 (995)	total: 31.6s	remaining: 0us
bestTest = 0.5064784976
bestIteration = 995
Shrink model to first 996 iterations.
Model fit: elapsed = 36.408 with test/NDCG@10 = 0.49691075612364327
-------------------- Model number 2 --------------------
Params:
	 - learning_rate: 0.05494343511669279
	 - depth: 3
	 - l2_leaf_reg: 13.285989180828237
	 - loss_function: YetiRank
	 - eval_metric: NDCG:top=10;type=Exp
	 - iterations: 1000
	 - bagging_temperature: 0.8937682339199968
	 - use_best_model: True
	 - min_data_in_leaf: 34
	 - verbose: 200
	 - grow_policy: Lossguide
	 - task_type: GPU


Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.2474886	best: 0.2474886 (0)	total: 40.3ms	remaining: 40.3s
200:	test: 0.4533229	best: 0.4533229 (200)	total: 5.23s	remaining: 20.8s
400:	test: 0.4703910	best: 0.4703910 (400)	total: 10.4s	remaining: 15.6s
600:	test: 0.4781788	best: 0.4781788 (600)	total: 15.6s	remaining: 10.4s
800:	test: 0.4836509	best: 0.4836509 (800)	total: 20.8s	remaining: 5.16s
999:	test: 0.4864931	best: 0.4866632 (995)	total: 25.9s	remaining: 0us
bestTest = 0.4866631814
bestIteration = 995
Shrink model to first 996 iterations.
Model fit: elapsed = 29.563 with test/NDCG@10 = 0.4781783851252525
-------------------- Model number 3 --------------------
Params:
	 - learning_rate: 0.04432113391500656
	 - depth: 6
	 - l2_leaf_reg: 26.861417022680314
	 - loss_function: YetiRank
	 - eval_metric: NDCG:top=10;type=Exp
	 - iterations: 1000
	 - bagging_temperature: 0.9890988281503088
	 - use_best_model: True
	 - min_data_in_leaf: 52
	 - verbose: 200
	 - grow_policy: Depthwise
	 - task_type: GPU


...

0:	test: 0.3004144	best: 0.3004144 (0)	total: 42ms	remaining: 41.9s
200:	test: 0.4785926	best: 0.4785926 (200)	total: 5.54s	remaining: 22s
400:	test: 0.4915215	best: 0.4915850 (395)	total: 11s	remaining: 16.5s
600:	test: 0.4959085	best: 0.4961803 (585)	total: 16.5s	remaining: 11s
800:	test: 0.4998378	best: 0.4999489 (765)	total: 21.9s	remaining: 5.45s
999:	test: 0.5014655	best: 0.5015344 (990)	total: 27.4s	remaining: 0us
bestTest = 0.5015344185
bestIteration = 990
Shrink model to first 991 iterations.
Model fit: elapsed = 31.644 with test/NDCG@10 = 0.4952033429226343
-------------------- Model number 24 --------------------
Params:
	 - learning_rate: 0.010513742381502343
	 - depth: 3
	 - l2_leaf_reg: 29.317324485388475
	 - loss_function: YetiRank
	 - eval_metric: NDCG:top=10;type=Exp
	 - iterations: 1000
	 - bagging_temperature: 0.8405953604943104
	 - use_best_model: True
	 - min_data_in_leaf: 11
	 - verbose: 200
	 - grow_policy: Depthwise
	 - task_type: GPU


Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.2727262	best: 0.2727262 (0)	total: 41.4ms	remaining: 41.3s
200:	test: 0.4130109	best: 0.4130109 (200)	total: 5.23s	remaining: 20.8s
400:	test: 0.4283201	best: 0.4283201 (400)	total: 10.4s	remaining: 15.6s
600:	test: 0.4362267	best: 0.4362267 (600)	total: 15.6s	remaining: 10.4s
800:	test: 0.4447355	best: 0.4447934 (795)	total: 20.8s	remaining: 5.17s
999:	test: 0.4507770	best: 0.4507994 (995)	total: 26.1s	remaining: 0us
bestTest = 0.4507994306
bestIteration = 995
Shrink model to first 996 iterations.
Model fit: elapsed = 29.684 with test/NDCG@10 = 0.44482621053836846
-------------------- Model number 25 --------------------
Params:
	 - learning_rate: 0.09769934773084338
	 - depth: 7
	 - l2_leaf_reg: 18.540520072606334
	 - loss_function: YetiRank
	 - eval_metric: NDCG:top=10;type=Exp
	 - iterations: 1000
	 - bagging_temperature: 0.9217790738194904
	 - use_best_model: True
	 - min_data_in_leaf: 63
	 - verbose: 200
	 - grow_policy: SymmetricTree
	 - task_type: GPU


Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.2922021	best: 0.2922021 (0)	total: 47.1ms	remaining: 47s
200:	test: 0.4737007	best: 0.4737007 (200)	total: 6.26s	remaining: 24.9s
400:	test: 0.4906803	best: 0.4908657 (390)	total: 12.5s	remaining: 18.6s
600:	test: 0.4963194	best: 0.4963194 (600)	total: 18.7s	remaining: 12.4s
800:	test: 0.4991567	best: 0.4991567 (800)	total: 24.9s	remaining: 6.17s
999:	test: 0.5015197	best: 0.5015678 (990)	total: 31.1s	remaining: 0us
bestTest = 0.5015678143
bestIteration = 990
Shrink model to first 991 iterations.
Model fit: elapsed = 33.558 with test/NDCG@10 = 0.49417442400052974


Попробуем еще немного подкрутить параметры:

In [67]:
def delta_params(n_iter, params, pool_train, pool_valid, X_test, y_test):
    np.random.seed(0)
    random.seed(0)
    best_score = 0
    best_model = None
    best_params = {}
    for i in range(n_iter):
        print(f'{"-"*20} Model number {i+1} {"-"*20}')
        params['learning_rate'] = params['learning_rate'] + np.random.normal(0, 0.01)
        params['depth'] = random.randint(3, 8)
        params['l2_leaf_reg'] = params['l2_leaf_reg'] + np.random.normal(0, 3)
        params['iterations'] = 5000
        params['bagging_temperature'] = params['bagging_temperature'] + np.random.uniform(0, 0.05)
        params['min_data_in_leaf'] = random.randint(40, 70)
        params['random_seed'] = 0
        
        print('Params:')
        for k,v in params.items():
            print(f'\t - {k}: {v}')
        model = catboost.CatBoost(params)
        start = timer()
        model.fit(pool_train, eval_set=pool_valid)
        y_pred = model.predict(X_test)
        score = utils.eval_metric(y_test, y_pred, 'NDCG:top=10;type=Exp', group_id=q_test)[0]
        if score > best_score:
            best_model = model
            best_score = score
            best_params = params
            
        elapsed = timer() - start
        print(f"Model fit: elapsed = {elapsed:.3f} with test/NDCG@10 = {score}")
        
    return best_model, best_params

In [68]:
best_model, best_params = delta_params(10, best_params, pool_train, pool_valid, X_test, y_test)

-------------------- Model number 1 --------------------
Params:
	 - learning_rate: 0.08614073307092152
	 - depth: 6
	 - l2_leaf_reg: 12.626200741741464
	 - loss_function: YetiRank
	 - eval_metric: NDCG:top=10;type=Exp
	 - iterations: 5000
	 - bagging_temperature: 0.8612477549433846
	 - use_best_model: True
	 - min_data_in_leaf: 64
	 - verbose: 200
	 - grow_policy: Depthwise
	 - task_type: GPU
	 - random_seed: 0


Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.3375796	best: 0.3375796 (0)	total: 51.5ms	remaining: 4m 17s
200:	test: 0.4921952	best: 0.4923191 (195)	total: 6.45s	remaining: 2m 34s
400:	test: 0.5028480	best: 0.5028480 (400)	total: 12.9s	remaining: 2m 27s
600:	test: 0.5077297	best: 0.5077297 (600)	total: 19s	remaining: 2m 19s
800:	test: 0.5093303	best: 0.5094286 (790)	total: 25.1s	remaining: 2m 11s
1000:	test: 0.5103990	best: 0.5105478 (995)	total: 31.2s	remaining: 2m 4s
1200:	test: 0.5114182	best: 0.5116128 (1105)	total: 37.3s	remaining: 1m 58s
1400:	test: 0.5120881	best: 0.5120881 (1400)	total: 43.5s	remaining: 1m 51s
1600:	test: 0.5125434	best: 0.5126763 (1595)	total: 49.6s	remaining: 1m 45s
1800:	test: 0.5121964	best: 0.5129800 (1655)	total: 55.6s	remaining: 1m 38s
2000:	test: 0.5132674	best: 0.5133006 (1995)	total: 1m 1s	remaining: 1m 32s
2200:	test: 0.5137279	best: 0.5137329 (2190)	total: 1m 7s	remaining: 1m 26s
2400:	test: 0.5141686	best: 0.5143104 (2385)	total: 1m 13s	remaining: 1m 19s
2600:	test: 0.5140475	best: 

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.3276757	best: 0.3276757 (0)	total: 52.3ms	remaining: 4m 21s
200:	test: 0.4842210	best: 0.4842210 (200)	total: 6.58s	remaining: 2m 37s
400:	test: 0.4982201	best: 0.4982201 (400)	total: 12.9s	remaining: 2m 28s
600:	test: 0.5031876	best: 0.5031876 (600)	total: 19.3s	remaining: 2m 20s
800:	test: 0.5056487	best: 0.5058083 (795)	total: 25.5s	remaining: 2m 13s
1000:	test: 0.5066456	best: 0.5068625 (980)	total: 31.6s	remaining: 2m 6s
1200:	test: 0.5088272	best: 0.5088272 (1200)	total: 37.8s	remaining: 1m 59s
1400:	test: 0.5095183	best: 0.5096302 (1385)	total: 43.9s	remaining: 1m 52s
1600:	test: 0.5103505	best: 0.5105913 (1560)	total: 50s	remaining: 1m 46s
1800:	test: 0.5105123	best: 0.5107526 (1665)	total: 56.2s	remaining: 1m 39s
2000:	test: 0.5112337	best: 0.5112337 (2000)	total: 1m 2s	remaining: 1m 33s
2200:	test: 0.5116254	best: 0.5117346 (2195)	total: 1m 8s	remaining: 1m 27s
2400:	test: 0.5121035	best: 0.5121035 (2400)	total: 1m 14s	remaining: 1m 20s
2600:	test: 0.5123819	best: 

...

0:	test: 0.3418100	best: 0.3418100 (0)	total: 55.3ms	remaining: 4m 36s
200:	test: 0.4961840	best: 0.4961840 (200)	total: 7.06s	remaining: 2m 48s
400:	test: 0.5039576	best: 0.5039576 (400)	total: 13.9s	remaining: 2m 39s
600:	test: 0.5075057	best: 0.5075773 (595)	total: 20.7s	remaining: 2m 31s
800:	test: 0.5097502	best: 0.5098805 (790)	total: 27.5s	remaining: 2m 24s
1000:	test: 0.5112347	best: 0.5114440 (990)	total: 34.2s	remaining: 2m 16s
1200:	test: 0.5114836	best: 0.5117357 (1155)	total: 40.9s	remaining: 2m 9s
1400:	test: 0.5122881	best: 0.5125866 (1370)	total: 47.5s	remaining: 2m 1s
1600:	test: 0.5132422	best: 0.5133051 (1580)	total: 54.1s	remaining: 1m 54s
1800:	test: 0.5133276	best: 0.5135654 (1760)	total: 1m	remaining: 1m 48s
2000:	test: 0.5140557	best: 0.5141585 (1995)	total: 1m 7s	remaining: 1m 41s
2200:	test: 0.5145911	best: 0.5146348 (2165)	total: 1m 13s	remaining: 1m 34s
2400:	test: 0.5150107	best: 0.5150107 (2400)	total: 1m 20s	remaining: 1m 27s
2600:	test: 0.5152080	best: 0

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.2856607	best: 0.2856607 (0)	total: 51.4ms	remaining: 4m 16s
200:	test: 0.4738129	best: 0.4738129 (200)	total: 5.67s	remaining: 2m 15s
400:	test: 0.4889841	best: 0.4889841 (400)	total: 11.2s	remaining: 2m 8s
600:	test: 0.4943728	best: 0.4943728 (600)	total: 16.7s	remaining: 2m 2s
800:	test: 0.4975194	best: 0.4975599 (795)	total: 22.1s	remaining: 1m 56s
1000:	test: 0.4990794	best: 0.4990794 (1000)	total: 27.6s	remaining: 1m 50s
1200:	test: 0.5010707	best: 0.5011461 (1190)	total: 33.1s	remaining: 1m 44s
1400:	test: 0.5022093	best: 0.5022093 (1400)	total: 38.5s	remaining: 1m 38s
1600:	test: 0.5028598	best: 0.5030052 (1550)	total: 43.9s	remaining: 1m 33s
1800:	test: 0.5031586	best: 0.5031955 (1730)	total: 49.3s	remaining: 1m 27s
2000:	test: 0.5038629	best: 0.5039730 (1975)	total: 54.7s	remaining: 1m 21s
2200:	test: 0.5042116	best: 0.5044181 (2185)	total: 1m	remaining: 1m 16s
2400:	test: 0.5045433	best: 0.5046016 (2390)	total: 1m 5s	remaining: 1m 10s
2600:	test: 0.5048865	best: 0.

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.2861461	best: 0.2861461 (0)	total: 45ms	remaining: 3m 45s
200:	test: 0.4717602	best: 0.4717602 (200)	total: 5.57s	remaining: 2m 13s
400:	test: 0.4871494	best: 0.4872725 (395)	total: 11.1s	remaining: 2m 6s
600:	test: 0.4934174	best: 0.4935417 (595)	total: 16.7s	remaining: 2m 2s
800:	test: 0.4967148	best: 0.4969612 (760)	total: 22.1s	remaining: 1m 55s
1000:	test: 0.4993367	best: 0.4993367 (1000)	total: 27.5s	remaining: 1m 49s
1200:	test: 0.5004984	best: 0.5007992 (1175)	total: 32.9s	remaining: 1m 44s
1400:	test: 0.5014116	best: 0.5015032 (1335)	total: 38.2s	remaining: 1m 38s
1600:	test: 0.5023831	best: 0.5023831 (1600)	total: 43.6s	remaining: 1m 32s
1800:	test: 0.5028948	best: 0.5028948 (1800)	total: 49s	remaining: 1m 27s
2000:	test: 0.5035877	best: 0.5036988 (1980)	total: 54.4s	remaining: 1m 21s
2200:	test: 0.5041100	best: 0.5041809 (2190)	total: 59.8s	remaining: 1m 16s
2400:	test: 0.5044551	best: 0.5046455 (2345)	total: 1m 5s	remaining: 1m 10s
2600:	test: 0.5051385	best: 0.5

In [72]:
best_model.get_params()

{'learning_rate': 0.10340626393986621,
 'depth': 7,
 'l2_leaf_reg': 17.272888261719643,
 'loss_function': 'YetiRank',
 'eval_metric': 'NDCG:top=10;type=Exp',
 'iterations': 5000,
 'bagging_temperature': 1.068369922077575,
 'use_best_model': True,
 'min_data_in_leaf': 68,
 'verbose': 200,
 'grow_policy': 'Depthwise',
 'task_type': 'GPU',
 'random_seed': 0}

### Сохраняем модель

In [75]:
with open('best_params.pkl', 'wb') as f:
    pickle.dump(best_model.get_params(), f)

### Загружаем модель

In [14]:
best_params = {}
with open('/kaggle/input/vk-catboost-params/other/catboost-vk-v3/1/best_params.pkl', 'rb') as f:
    best_params = pickle.load(f)

In [15]:
best_params

{'learning_rate': 0.10340626393986621,
 'depth': 7,
 'l2_leaf_reg': 17.272888261719643,
 'loss_function': 'YetiRank',
 'eval_metric': 'NDCG:top=10;type=Exp',
 'iterations': 5000,
 'bagging_temperature': 1.068369922077575,
 'use_best_model': True,
 'min_data_in_leaf': 68,
 'verbose': 200,
 'grow_policy': 'Depthwise',
 'task_type': 'GPU',
 'random_seed': 0}

### Проверяем качество + подберем оптимальное число итераций.

In [21]:
best_params['iterations'] = 10_000

model = catboost.CatBoost(best_params)
start = timer()
model.fit(pool_train, eval_set=pool_valid)
y_pred = model.predict(X_test)
score = utils.eval_metric(y_test, y_pred, 'NDCG:top=10;type=Exp', group_id=q_test)[0]

elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f} with test/NDCG@10 = {score}")

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.3418100	best: 0.3418100 (0)	total: 22.6s	remaining: 2d 14h 44m 8s
200:	test: 0.4955327	best: 0.4955327 (200)	total: 29.6s	remaining: 24m 2s
400:	test: 0.5037397	best: 0.5037397 (400)	total: 36.5s	remaining: 14m 34s
600:	test: 0.5077113	best: 0.5077300 (595)	total: 43.3s	remaining: 11m 17s
800:	test: 0.5097561	best: 0.5098904 (795)	total: 50s	remaining: 9m 33s
1000:	test: 0.5112044	best: 0.5112044 (1000)	total: 56.5s	remaining: 8m 27s
1200:	test: 0.5116722	best: 0.5118589 (1160)	total: 1m 3s	remaining: 7m 42s
1400:	test: 0.5124102	best: 0.5124102 (1400)	total: 1m 9s	remaining: 7m 7s
1600:	test: 0.5128889	best: 0.5128889 (1600)	total: 1m 16s	remaining: 6m 40s
1800:	test: 0.5134003	best: 0.5134003 (1800)	total: 1m 22s	remaining: 6m 17s
2000:	test: 0.5140772	best: 0.5144158 (1965)	total: 1m 29s	remaining: 5m 57s
2200:	test: 0.5147466	best: 0.5147466 (2200)	total: 1m 36s	remaining: 5m 40s
2400:	test: 0.5149042	best: 0.5150870 (2315)	total: 1m 42s	remaining: 5m 24s
2600:	test: 0.5

Оптимальным числом итераций является 8615. NDSG@10 на тесте 0.51229.

## Проведем feature selection.

### Baseline 1000 итераций 100% признаков

In [18]:
best_params['iterations'] = 1000
model = catboost.CatBoost(best_params)
start = timer()
model.fit(pool_train, eval_set=pool_valid)
y_pred = model.predict(X_test)
score = utils.eval_metric(y_test, y_pred, 'NDCG:top=10;type=Exp', group_id=q_test)[0]

elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f} with test/NDCG@10 = {score}")

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.3418100	best: 0.3418100 (0)	total: 21.6s	remaining: 5h 59m 31s
200:	test: 0.4961840	best: 0.4961840 (200)	total: 28.7s	remaining: 1m 53s
400:	test: 0.5039576	best: 0.5039576 (400)	total: 35.6s	remaining: 53.2s
600:	test: 0.5075725	best: 0.5075725 (600)	total: 42.5s	remaining: 28.2s
800:	test: 0.5098837	best: 0.5098837 (800)	total: 49.3s	remaining: 12.3s
999:	test: 0.5107292	best: 0.5110378 (975)	total: 56s	remaining: 0us
bestTest = 0.5110378479
bestIteration = 975
Shrink model to first 976 iterations.
Model fit: elapsed = 62.137 with test/NDCG@10 = 0.5038919620562503


### 75% признаков

In [19]:
result75 = model.select_features(
                pool_train,
                eval_set=pool_valid,
                features_for_select=np.arange(X_train.shape[1]),
                num_features_to_select = 3 * X_train.shape[1] // 4,
                algorithm='RecursiveByShapValues',
                steps=1,
                shap_calc_type='Approximate',
                train_final_model=False)['selected_features']

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


Step #1 out of 1
0:	test: 0.3418100	best: 0.3418100 (0)	total: 55.5ms	remaining: 55.5s
200:	test: 0.4961840	best: 0.4961840 (200)	total: 7.2s	remaining: 28.6s
400:	test: 0.5039576	best: 0.5039576 (400)	total: 14.2s	remaining: 21.2s
600:	test: 0.5075707	best: 0.5075707 (600)	total: 21.1s	remaining: 14s
800:	test: 0.5097039	best: 0.5097896 (790)	total: 27.9s	remaining: 6.93s
999:	test: 0.5112119	best: 0.5112305 (975)	total: 34.6s	remaining: 0us
bestTest = 0.5112304998
bestIteration = 975
Shrink model to first 976 iterations.
Feature #121 eliminated
Feature #90 eliminated
Feature #94 eliminated
Feature #106 eliminated
Feature #76 eliminated
Feature #44 eliminated
Feature #43 eliminated
Feature #0 eliminated
Feature #51 eliminated
Feature #27 eliminated
Feature #31 eliminated
Feature #61 eliminated
Feature #33 eliminated
Feature #28 eliminated
Feature #96 eliminated
Feature #97 eliminated
Feature #98 eliminated
Feature #99 eliminated
Feature #36 eliminated
Feature #104 eliminated
Feature #

Проверим качество:

In [20]:
pool_train_selected = catboost.Pool(data=X_train[:, result75], label=y_train, group_id=q_train)
pool_valid_selected = catboost.Pool(data=X_valid[:, result75], label=y_valid, group_id=q_valid)
X_test_selected = X_test[:, result75]
X_test_selected.shape

(241521, 102)

In [21]:
best_params['iterations'] = 10_000
model1 = catboost.CatBoost(best_params)
start = timer()
model1.fit(pool_train_selected, eval_set=pool_valid_selected)
y_pred = model1.predict(X_test_selected)
score = utils.eval_metric(y_test, y_pred, 'NDCG:top=10;type=Exp', group_id=q_test)[0]

elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f} with test/NDCG@10 = {score}")

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.3401737	best: 0.3401737 (0)	total: 49.9ms	remaining: 8m 18s
200:	test: 0.4964417	best: 0.4964417 (200)	total: 6.86s	remaining: 5m 34s
400:	test: 0.5042694	best: 0.5042694 (400)	total: 13.5s	remaining: 5m 23s
600:	test: 0.5088452	best: 0.5088452 (600)	total: 20.1s	remaining: 5m 14s
800:	test: 0.5100050	best: 0.5100050 (800)	total: 26.6s	remaining: 5m 5s
1000:	test: 0.5108595	best: 0.5110758 (990)	total: 33.1s	remaining: 4m 57s
1200:	test: 0.5125296	best: 0.5126890 (1175)	total: 39.6s	remaining: 4m 50s
1400:	test: 0.5132054	best: 0.5134765 (1305)	total: 46s	remaining: 4m 42s
1600:	test: 0.5136946	best: 0.5140209 (1530)	total: 52.6s	remaining: 4m 36s
1800:	test: 0.5143327	best: 0.5143907 (1785)	total: 59.1s	remaining: 4m 29s
2000:	test: 0.5144962	best: 0.5144983 (1985)	total: 1m 5s	remaining: 4m 22s
2200:	test: 0.5152687	best: 0.5152687 (2200)	total: 1m 12s	remaining: 4m 15s
2400:	test: 0.5152965	best: 0.5154664 (2230)	total: 1m 18s	remaining: 4m 8s
2600:	test: 0.5156696	best: 

### 50% признаков

In [22]:
result50 = model.select_features(
                pool_train,
                eval_set=pool_valid,
                features_for_select=np.arange(X_train.shape[1]),
                num_features_to_select = X_train.shape[1] // 2,
                algorithm='RecursiveByShapValues',
                steps=1,
                shap_calc_type='Approximate',
                train_final_model=False)['selected_features']

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


Step #1 out of 1
0:	test: 0.3418100	best: 0.3418100 (0)	total: 55.2ms	remaining: 55.1s
200:	test: 0.4961840	best: 0.4961840 (200)	total: 7.13s	remaining: 28.3s
400:	test: 0.5039576	best: 0.5039576 (400)	total: 14.1s	remaining: 21.1s
600:	test: 0.5075725	best: 0.5075725 (600)	total: 20.9s	remaining: 13.9s
800:	test: 0.5095598	best: 0.5096511 (785)	total: 27.6s	remaining: 6.87s
999:	test: 0.5109636	best: 0.5111249 (945)	total: 34.3s	remaining: 0us
bestTest = 0.5111248997
bestIteration = 945
Shrink model to first 946 iterations.
Feature #100 eliminated
Feature #83 eliminated
Feature #46 eliminated
Feature #22 eliminated
Feature #76 eliminated
Feature #95 eliminated
Feature #81 eliminated
Feature #61 eliminated
Feature #37 eliminated
Feature #79 eliminated
Feature #28 eliminated
Feature #26 eliminated
Feature #31 eliminated
Feature #96 eliminated
Feature #97 eliminated
Feature #98 eliminated
Feature #99 eliminated
Feature #56 eliminated
Feature #0 eliminated
Feature #33 eliminated
Feature 

In [23]:
pool_train_selected = catboost.Pool(data=X_train[:, result50], label=y_train, group_id=q_train)
pool_valid_selected = catboost.Pool(data=X_valid[:, result50], label=y_valid, group_id=q_valid)
X_test_selected = X_test[:, result50]
X_test_selected.shape

(241521, 68)

In [24]:
best_params['iterations'] = 10_000
model1 = catboost.CatBoost(best_params)
start = timer()
model1.fit(pool_train_selected, eval_set=pool_valid_selected)
y_pred = model1.predict(X_test_selected)
score = utils.eval_metric(y_test, y_pred, 'NDCG:top=10;type=Exp', group_id=q_test)[0]

elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f} with test/NDCG@10 = {score}")

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.3447758	best: 0.3447758 (0)	total: 50.5ms	remaining: 8m 25s
200:	test: 0.4945795	best: 0.4945795 (200)	total: 6.58s	remaining: 5m 20s
400:	test: 0.5045132	best: 0.5045763 (380)	total: 13s	remaining: 5m 10s
600:	test: 0.5078777	best: 0.5078777 (600)	total: 19.4s	remaining: 5m 2s
800:	test: 0.5097308	best: 0.5099568 (780)	total: 25.7s	remaining: 4m 55s
1000:	test: 0.5108019	best: 0.5108547 (940)	total: 32s	remaining: 4m 47s
1200:	test: 0.5111183	best: 0.5111188 (1045)	total: 38.3s	remaining: 4m 40s
1400:	test: 0.5118976	best: 0.5118976 (1400)	total: 44.6s	remaining: 4m 33s
1600:	test: 0.5127989	best: 0.5127989 (1600)	total: 50.9s	remaining: 4m 26s
1800:	test: 0.5129098	best: 0.5133581 (1755)	total: 57.3s	remaining: 4m 20s
2000:	test: 0.5132511	best: 0.5133581 (1755)	total: 1m 3s	remaining: 4m 13s
2200:	test: 0.5137977	best: 0.5139482 (2190)	total: 1m 9s	remaining: 4m 6s
2400:	test: 0.5141027	best: 0.5142811 (2350)	total: 1m 15s	remaining: 4m
2600:	test: 0.5140284	best: 0.51436

Для дальнейших экспериментов, чтобы хватало оперативной памяти, сохраним половину отобранных признаков.

In [25]:
with open('best_indeces.pkl', 'wb') as f:
    pickle.dump(result50, f)

In [27]:
result50[:20]

[1, 2, 5, 7, 10, 11, 12, 13, 14, 17, 19, 20, 24, 29, 35, 40, 45, 47, 48, 49]

In [28]:
with open('/kaggle/input/vk-catboost-params/other/half_features/1/best_indeces.pkl', 'rb') as f:
    result50 = pickle.load(f)
result50[:20]

[1, 2, 5, 7, 10, 11, 12, 13, 14, 17, 19, 20, 24, 29, 35, 40, 45, 47, 48, 49]

In [29]:
X_train_selected = X_train[:, result50]
X_valid_selected = X_valid[:, result50]
X_test_selected = X_test[:, result50]
X_train_selected.shape, X_valid_selected.shape, X_test_selected.shape

((723412, 68), (235259, 68), (241521, 68))

Видно, что качество ухудшилось, значит оставим 75% признаков.

### Попробуем добавить примитивную кластеризацию.

За признаки возьмем растояния до центров классов.

In [16]:
from sklearn.cluster import KMeans

In [17]:
def euclidean_distance(X: np.array, Y: np.array) -> np.array:
    n, m = X.shape[0], Y.shape[0]

    Z = np.dot(X, Y.T)
    X = np.einsum('ij,ij->i', X, X).reshape(n, 1)
    Y = np.einsum('ij,ij->i', Y, Y)

    X = np.repeat(X, repeats=m, axis=1)
    Y = np.tile(Y, (n, 1))

    return (X - 2 * Z + Y) 

In [18]:
clusterts_dict = {}
for n_clusters in [64, 128, 256, 512, 1024]:
    print(f'{"-"*20} KMeans k = {n_clusters} {"-"*20}')
    kmeans = KMeans(n_clusters=n_clusters, n_init=3, verbose=1, random_state=0)
    kmeans.fit(X_train)
    
    clusters = kmeans.cluster_centers_
    clusterts_dict[n_clusters] = clusters
    
    X_train_distances = euclidean_distance(X_train, clusters)
    X_valid_distances = euclidean_distance(X_valid, clusters)
    pool_train_distances = catboost.Pool(data=X_train_distances, label=y_train, group_id=q_train)
    pool_valid_distances = catboost.Pool(data=X_valid_distances, label=y_valid, group_id=q_valid)
    
    X_test_distances = euclidean_distance(X_test, clusters)

    best_params['iterations'] = 1000
    model = catboost.CatBoost(best_params)
    start = timer()
    model.fit(pool_train_distances, eval_set=pool_valid_distances)
    y_pred = model.predict(X_test_distances)
    score = utils.eval_metric(y_test, y_pred, 'NDCG:top=10;type=Exp', group_id=q_test)[0]

-------------------- KMeans k = 64 --------------------
Initialization complete
Iteration 0, inertia 1025100379848704.0.
Iteration 1, inertia 889565170630656.0.
Iteration 2, inertia 871214318878720.0.
Iteration 3, inertia 861057392312320.0.
Iteration 4, inertia 860232221720576.0.
Iteration 5, inertia 859795208798208.0.
Iteration 6, inertia 859429733924864.0.
Iteration 7, inertia 859300012490752.0.
Converged at iteration 7: center shift 23859972.0 within tolerance 50386282.0864.
Initialization complete
Iteration 0, inertia 1032960371326976.0.
Iteration 1, inertia 851611349942272.0.
Iteration 2, inertia 838175014518784.0.
Iteration 3, inertia 832582363119616.0.
Iteration 4, inertia 830094905966592.0.
Iteration 5, inertia 829118606213120.0.
Iteration 6, inertia 828359739179008.0.
Iteration 7, inertia 827901586964480.0.
Iteration 8, inertia 827615434768384.0.
Iteration 9, inertia 827485042245632.0.
Iteration 10, inertia 827299754672128.0.
Iteration 11, inertia 827198218960896.0.
Iteration 

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.2610270	best: 0.2610270 (0)	total: 16.8s	remaining: 4h 39m 52s
200:	test: 0.3409517	best: 0.3409517 (200)	total: 23.2s	remaining: 1m 32s
400:	test: 0.3487591	best: 0.3487591 (400)	total: 29.4s	remaining: 44s
600:	test: 0.3527338	best: 0.3527338 (600)	total: 35.7s	remaining: 23.7s
800:	test: 0.3542855	best: 0.3545040 (775)	total: 41.8s	remaining: 10.4s
999:	test: 0.3552373	best: 0.3555762 (975)	total: 48s	remaining: 0us
bestTest = 0.3555761692
bestIteration = 975
Shrink model to first 976 iterations.
-------------------- KMeans k = 128 --------------------
Initialization complete
Iteration 0, inertia 146688836632576.0.
Iteration 1, inertia 122472192016384.0.
Iteration 2, inertia 119731407290368.0.
Iteration 3, inertia 118662698631168.0.
Iteration 4, inertia 118193616060416.0.
Iteration 5, inertia 117906364956672.0.
Converged at iteration 5: center shift 40671312.0 within tolerance 50386282.0864.
Initialization complete
Iteration 0, inertia 150017352400896.0.
Iteration 1, iner

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.2484289	best: 0.2484289 (0)	total: 53.3ms	remaining: 53.2s
200:	test: 0.3522061	best: 0.3522061 (200)	total: 6.86s	remaining: 27.3s
400:	test: 0.3607850	best: 0.3608647 (395)	total: 13.5s	remaining: 20.1s
600:	test: 0.3645366	best: 0.3645366 (600)	total: 20s	remaining: 13.3s
800:	test: 0.3660644	best: 0.3661860 (780)	total: 26.6s	remaining: 6.6s
999:	test: 0.3670219	best: 0.3670389 (995)	total: 33.1s	remaining: 0us
bestTest = 0.3670389439
bestIteration = 995
Shrink model to first 996 iterations.
-------------------- KMeans k = 256 --------------------
Initialization complete
Iteration 0, inertia 42652670099456.0.
Iteration 1, inertia 35315779633152.0.
Iteration 2, inertia 34611157532672.0.
Iteration 3, inertia 34388700037120.0.
Iteration 4, inertia 34258567561216.0.
Iteration 5, inertia 34169860128768.0.
Iteration 6, inertia 34104483512320.0.
Iteration 7, inertia 34054040715264.0.
Converged at iteration 7: center shift 35053884.0 within tolerance 50386282.0864.
Initializatio

...

0:	test: 0.2491225	best: 0.2491225 (0)	total: 60.3ms	remaining: 1m
200:	test: 0.3532263	best: 0.3532263 (200)	total: 7.74s	remaining: 30.8s
400:	test: 0.3604871	best: 0.3606508 (395)	total: 15.3s	remaining: 22.9s
600:	test: 0.3650180	best: 0.3652990 (575)	total: 22.7s	remaining: 15.1s
800:	test: 0.3665205	best: 0.3666187 (795)	total: 30.1s	remaining: 7.48s
999:	test: 0.3674001	best: 0.3675733 (925)	total: 37.4s	remaining: 0us
bestTest = 0.367573278
bestIteration = 925
Shrink model to first 926 iterations.
-------------------- KMeans k = 512 --------------------
Initialization complete
Iteration 0, inertia 18164362510336.0.
Iteration 1, inertia 15175381942272.0.
Iteration 2, inertia 14800046260224.0.
Iteration 3, inertia 14632655781888.0.
Iteration 4, inertia 14522331955200.0.
Iteration 5, inertia 14455513546752.0.
Iteration 6, inertia 14408683094016.0.
Iteration 7, inertia 14373696307200.0.
Converged at iteration 7: center shift 34070800.0 within tolerance 50386282.0864.
Initialization

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.2578344	best: 0.2578344 (0)	total: 72.9ms	remaining: 1m 12s
200:	test: 0.3526963	best: 0.3526963 (200)	total: 9.45s	remaining: 37.6s
400:	test: 0.3610444	best: 0.3610444 (400)	total: 18.3s	remaining: 27.3s
600:	test: 0.3666168	best: 0.3666168 (600)	total: 27s	remaining: 18s
800:	test: 0.3693729	best: 0.3693729 (800)	total: 35.8s	remaining: 8.9s
999:	test: 0.3703651	best: 0.3704395 (960)	total: 44.6s	remaining: 0us
bestTest = 0.3704394685
bestIteration = 960
Shrink model to first 961 iterations.
-------------------- KMeans k = 1024 --------------------
Initialization complete
Iteration 0, inertia 9092598857728.0.
Iteration 1, inertia 7665983946752.0.
Iteration 2, inertia 7473280843776.0.
Iteration 3, inertia 7388425879552.0.
Iteration 4, inertia 7329665777664.0.
Iteration 5, inertia 7291101773824.0.
Iteration 6, inertia 7265859403776.0.
Iteration 7, inertia 7245778649088.0.
Iteration 8, inertia 7228815835136.0.
Iteration 9, inertia 7215057993728.0.
Iteration 10, inertia 72044

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.2545986	best: 0.2545986 (0)	total: 91ms	remaining: 1m 30s
200:	test: 0.3553808	best: 0.3553808 (200)	total: 12.3s	remaining: 48.9s
400:	test: 0.3611496	best: 0.3611496 (400)	total: 23.9s	remaining: 35.6s
600:	test: 0.3665970	best: 0.3665970 (600)	total: 35.3s	remaining: 23.4s
800:	test: 0.3686452	best: 0.3687920 (780)	total: 46.8s	remaining: 11.6s
999:	test: 0.3704560	best: 0.3704560 (999)	total: 58.2s	remaining: 0us
bestTest = 0.3704560336
bestIteration = 999


In [37]:
clusterts_dict[128].shape

(128, 136)

In [38]:
X_train_distances = euclidean_distance(X_train, clusterts_dict[128])
X_valid_distances = euclidean_distance(X_valid, clusterts_dict[128])
X_train_distances.shape, X_valid_distances.shape

((723412, 128), (235259, 128))

In [39]:
pool_train_distances = catboost.Pool(
            data=np.hstack([X_train, X_train_distances]), 
            label=y_train, group_id=q_train
            )
pool_valid_distances = catboost.Pool(
            data=np.hstack([X_valid, X_valid_distances]),
            label=y_valid, group_id=q_valid
            )

X_test_distances = np.hstack(
    [X_test, euclidean_distance(X_test, clusterts_dict[128])]
   )

Проверим, произойдет ли прирост качества, и заодно найдем оптимальное число итераций.

In [40]:
best_params['iterations'] = 15_000
best_params['verbose'] = 1000
model = catboost.CatBoost(best_params)
start = timer()
model.fit(pool_train_distances, eval_set=pool_valid_distances)
y_pred = model.predict(X_test_distances)
score = utils.eval_metric(y_test, y_pred, 'NDCG:top=10;type=Exp', group_id=q_test)[0]

elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f} with test/NDCG@10 = {score}")

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.3448433	best: 0.3448433 (0)	total: 63.2ms	remaining: 15m 47s
1000:	test: 0.5103019	best: 0.5107394 (880)	total: 39.6s	remaining: 9m 14s
2000:	test: 0.5137487	best: 0.5140701 (1975)	total: 1m 17s	remaining: 8m 24s
3000:	test: 0.5151977	best: 0.5153651 (2950)	total: 1m 55s	remaining: 7m 42s
4000:	test: 0.5170166	best: 0.5170166 (4000)	total: 2m 33s	remaining: 7m 3s
5000:	test: 0.5177565	best: 0.5179227 (4980)	total: 3m 12s	remaining: 6m 24s
6000:	test: 0.5180898	best: 0.5184100 (5630)	total: 3m 50s	remaining: 5m 45s
7000:	test: 0.5186503	best: 0.5190027 (6960)	total: 4m 28s	remaining: 5m 7s
8000:	test: 0.5184952	best: 0.5190389 (7360)	total: 5m 7s	remaining: 4m 28s
9000:	test: 0.5181638	best: 0.5190389 (7360)	total: 5m 45s	remaining: 3m 50s
10000:	test: 0.5178333	best: 0.5190389 (7360)	total: 6m 24s	remaining: 3m 12s
11000:	test: 0.5180812	best: 0.5190389 (7360)	total: 7m 2s	remaining: 2m 33s
12000:	test: 0.5179128	best: 0.5190389 (7360)	total: 7m 41s	remaining: 1m 55s
13000:	

Итого, перебрав вручную k=64, 256, 128, 512 - оптимальное качество на валидации достигается при k=128. Но при этом качество на тесте ухудшается. Таким образом метод не дает никаких полезных результатов.

## Добавим вторые степени признаков.

In [30]:
X_train_with_squares = (X_train_selected[:,:,None]*X_train_selected[:,None,:]).reshape(X_train_selected.shape[0],-1)
X_valid_with_squares = (X_valid_selected[:,:,None]*X_valid_selected[:,None,:]).reshape(X_valid_selected.shape[0],-1)
X_test_with_squares = (X_test_selected[:,:,None]*X_test_selected[:,None,:]).reshape(X_test_selected.shape[0],-1)
X_train_with_squares.shape, X_valid_with_squares.shape, X_test_with_squares.shape

((723412, 4624), (235259, 4624), (241521, 4624))

In [32]:
pool_train_squares = catboost.Pool(data=X_train_with_squares, label=y_train, group_id=q_train)
pool_valid_squares = catboost.Pool(data=X_valid_with_squares, label=y_valid, group_id=q_valid)

In [33]:
best_params['iterations'] = 10_000
model1 = catboost.CatBoost(best_params)
start = timer()
model1.fit(pool_train_squares, eval_set=pool_valid_squares)
y_pred = model1.predict(X_test_with_squares)
score = utils.eval_metric(y_test, y_pred, 'NDCG:top=10;type=Exp', group_id=q_test)[0]

elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f} with test/NDCG@10 = {score}")

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.3430107	best: 0.3430107 (0)	total: 264ms	remaining: 44m 4s
200:	test: 0.4987203	best: 0.4987203 (200)	total: 38.5s	remaining: 31m 19s
400:	test: 0.5062413	best: 0.5062413 (400)	total: 1m 11s	remaining: 28m 38s
600:	test: 0.5100462	best: 0.5100462 (600)	total: 1m 42s	remaining: 26m 50s
800:	test: 0.5103095	best: 0.5111176 (780)	total: 2m 13s	remaining: 25m 35s
1000:	test: 0.5113879	best: 0.5115883 (920)	total: 2m 42s	remaining: 24m 23s
1200:	test: 0.5122962	best: 0.5125612 (1170)	total: 3m 12s	remaining: 23m 28s
1400:	test: 0.5134747	best: 0.5137121 (1370)	total: 3m 41s	remaining: 22m 39s
1600:	test: 0.5138865	best: 0.5140083 (1535)	total: 4m 11s	remaining: 21m 56s
1800:	test: 0.5144862	best: 0.5147300 (1760)	total: 4m 39s	remaining: 21m 14s
2000:	test: 0.5149401	best: 0.5152340 (1980)	total: 5m 9s	remaining: 20m 36s
2200:	test: 0.5149205	best: 0.5152340 (1980)	total: 5m 39s	remaining: 20m 2s
2400:	test: 0.5154017	best: 0.5154618 (2355)	total: 6m 8s	remaining: 19m 27s
2600:	t

Как видно, такой подход не дает прироста в качестве, но требует большое количество памяти (~29 гб) и сильно замедляет обучение. Остановимся на варианте все признаки + кластеризация.

***Итого: ничего не улучшило ситуацию.***

# Итоговый (грустный) результат

In [47]:
best_params

{'learning_rate': 0.10340626393986621,
 'depth': 7,
 'l2_leaf_reg': 17.272888261719643,
 'loss_function': 'YetiRank',
 'eval_metric': 'NDCG:top=10;type=Exp',
 'iterations': 15000,
 'bagging_temperature': 1.068369922077575,
 'use_best_model': True,
 'min_data_in_leaf': 68,
 'verbose': 1000,
 'grow_policy': 'Depthwise',
 'task_type': 'GPU',
 'random_seed': 0}

In [53]:
class Model:
    def __init__(self, device='CPU'):
        params = {
            'learning_rate': 0.10340626393986621,
             'depth': 7,
             'l2_leaf_reg': 17.272888261719643,
             'loss_function': 'YetiRank',
             'eval_metric': 'NDCG:top=10;type=Exp',
             'iterations': 8500,
             'bagging_temperature': 1.068369922077575,
             'use_best_model': False,
             'min_data_in_leaf': 68,
             'verbose': 500,
             'grow_policy': 'Depthwise',
             'task_type': device,
             'random_seed': 0
        }
        self.model = catboost.CatBoost(params)

    def _to_catboost_dataset(self, df):
        y = df['label'].to_numpy()                       # Label: [0-4]
        q = df['qid'].to_numpy().astype('uint32')        # Query Id
        X = df.drop(columns=['label', 'qid']).to_numpy() # 136 features
        return (X, y, q)


    def fit(self, df_train, df_valid):
        X_train, y_train, q_train = self._to_catboost_dataset(df_train)
        X_valid, y_valid, q_valid = self._to_catboost_dataset(df_valid)
        X_train_valid = np.r_[X_train, X_valid]
        y_train_valid = np.r_[y_train, y_valid]
        q_train_valid = np.r_[q_train, q_valid]
        
        pool_train = catboost.Pool(data=X_train_valid, 
                                   label=y_train_valid, 
                                   group_id=q_train_valid
                                   )
        self.model.fit(pool_train)


    def predict(self, df_test):
        X_test, y_test, q_test = to_catboost_dataset(df_test)
        return self.model.predict(X_test)

Создадим и применим модель:

In [55]:
# Create model
model = Model('CPU')

# Fit
start = timer()
model.fit(df_train, df_valid)
elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f}")

# Predict
y_hat_test = model.predict(df_test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	total: 44.5ms	remaining: 6m 18s
500:	total: 18.8s	remaining: 5m
1000:	total: 36.8s	remaining: 4m 35s
1500:	total: 54.5s	remaining: 4m 14s
2000:	total: 1m 12s	remaining: 3m 54s
2500:	total: 1m 29s	remaining: 3m 35s
3000:	total: 1m 47s	remaining: 3m 17s
3500:	total: 2m 5s	remaining: 2m 58s
4000:	total: 2m 23s	remaining: 2m 40s
4500:	total: 2m 40s	remaining: 2m 22s
5000:	total: 2m 58s	remaining: 2m 5s
5500:	total: 3m 16s	remaining: 1m 47s
6000:	total: 3m 34s	remaining: 1m 29s
6500:	total: 3m 52s	remaining: 1m 11s
7000:	total: 4m 10s	remaining: 53.6s
7500:	total: 4m 28s	remaining: 35.7s
8000:	total: 4m 46s	remaining: 17.8s
8499:	total: 5m 3s	remaining: 0us
Model fit: elapsed = 309.156
Predicted: y_hat_test.shape = (241521,)


In [56]:
y_hat_test = model.predict(df_test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

Predicted: y_hat_test.shape = (241521,)


Теперь, имея предикты, можно посчитать метрики качества:

In [59]:
def compute_metrics(y_true, y_hat, q):
    # List of metrics to evaluate
    eval_metrics = ['NDCG:top=10;type=Exp']
    
    for eval_metric in eval_metrics:
        scores = utils.eval_metric(y_true, y_hat, eval_metric, group_id=q)
    
        # Print scores
        print(f"metric = {eval_metric} score = {scores[0]:.3f}")

# Get test targets and groups
y_test = df_test['label'].to_numpy()
q_test = df_test['qid'].to_numpy().astype('uint32')
    
# Compute metrics on test
compute_metrics(y_test, y_hat_test, q_test)

metric = NDCG:top=10;type=Exp score = 0.512


Ожидаем, что ваша модель покажет результаты выше бейзлайна!